In [ ]:
import os
hf_token = "HUGGINGFACE TOKEN HERE"
os.environ["HUGGINGFACEHUB_API_TOKEN"]=hf_token
os.environ["HF_TOKEN"]=hf_token
os.environ['HF_HOME'] = 'YOUR_HOME_DIR/.cache/huggingface/'
os.environ['TRANSFOMERS_CACHE'] = 'YOUR_HOME_DIR/.cache/huggingface/'
import transformers

In [ ]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from transformers import BitsAndBytesConfig
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
from llama_index.core import Settings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from llama_index.llms.huggingface import HuggingFaceLLM
from peft import PeftModel

EMBED_MODEL = HuggingFaceEmbedding(model_name="abhinand/MedEmbed-large-v0.1")
model_id = "epfl-llm/meditron-7b"

model = AutoModelForCausalLM.from_pretrained(model_id,
    device_map="auto",
    token=hf_token,
    torch_dtype=torch.float16,
)

#GEN_MODEL = HuggingFaceLLM(model=model,tokenizer_name=model_id,
#    device_map="auto",
#    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
#)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

Settings.tokenzier = tokenizer
#Settings.llm = model

embed_dim = len(EMBED_MODEL.get_text_embedding("hi"))
print("Embed dim:", embed_dim)

In [ ]:
from pyarrow.parquet import ParquetDataset
import time
dataset = ParquetDataset(f"YOUR_HOME_DIR/datasets/deepseek_COT_raft/val.parquet")
pds = dataset.read().to_pandas()

In [ ]:
for idx, row in pds.iterrows():
    e = row.to_dict()
    query = e["input"]
#    input_text = tokenizer.apply_chat_template([{"role": "user", "content": query}], tokenize=False)
    inputs = tokenizer(query, return_tensors="pt")
    num_tokens = inputs["input_ids"].shape[1]
    print("num_tokens", num_tokens)
    inputs.to("cuda")
    output = model.generate(**inputs, max_new_tokens=4096)
    
    # Decode and print result
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)